In [1]:
using SpecialFunctions
using QuantumOptics

function fidelity(ρ, σ)
    tr(sqrt(sqrt(ρ) * σ * sqrt(ρ)))^(1/2)
end

function R(n::Int64, m::Int64, ρ::Float64)
    if (n - m) % 2 != 0
        0
    else
        function summand(k)
            n = big(n)
            k = big(k)
            (-1)^k * factorial(n-k)/(factorial(k)*factorial(Int((n+m)/2) - k)*factorial(Int((n-m)/2) - k))*(ρ)^(n-2*k)
        end
        mapreduce(summand, +, Array(range(0, stop=Int((n-m)/2), step=1)))
    end
end

function sample_across_unit_disk(f, discretization)
    x = y = Array(range(-1, 1, length=discretization))
    z = Array{typeof(f(0.,0.))}(UndefInitializer(), length(x), length(y))
    for (i, xx) in enumerate(x)
        for (j, yy) in enumerate(y)
            if xx^2 + yy^2 > 1
                if typeof(f(0.,0.)) == Tuple{Float64,Float64}
                    z[i, j] = (0.,0.)
                else
                    z[i, j] = 0.
                end
            else
                ρ = sqrt(xx^2 + yy^2)
                ϕ = atan(yy, xx)
                res = f(ρ, ϕ)
                z[i, j] = res
            end
        end
    end
    x, y, z
end

function unpack_zernike(zernike_coefficients_even, zernike_coefficients_odd, ρ, ω, t)
    """Each Zernike array is of arbitrary length, but they are ordered by order. The first is Z00 to Zn0 where
    n is the number of 0th order zernike polynomials being considered."""
    Pevens = []
    eventuples = []
    for (n, zeven) in enumerate(zernike_coefficients_even)
        neven = n - 1
        push!(Pevens, [zeven[i] * R(neven, i-1, ρ) for i in Array(range(1, length(zeven), step=1))])
    end
    for i in range(1, length(Pevens), step=1)
        rotationeven = - (i - 1) * ω * t
        total = 0
        for peven in Pevens
            if length(peven) >= i
                total += peven[i]
            end
        end    
        push!(eventuples, (i - 1, total, rotationeven))
    end

    Podds = []
    oddtuples = []
    for (n, zodd) in enumerate(zernike_coefficients_odd)
        nodd = n - 1
        push!(Podds, [zodd[i] * R(nodd, i-1, ρ) for i in Array(range(1, length(zodd), step=1))])
    end
    for i in range(1, length(Podds), step=1)
        rotationodd = - (i - 1) * ω * t
        total = 0
        for podd in Podds
            if length(podd) >= i
                total += podd[i]
            end
        end    
        push!(oddtuples, (i - 1, total, rotationodd))
    end
    eventuples, oddtuples
end

function infidelity_across_disk(F1, F2)
    function infidelity_polar(ρ, ϕ)
        ψ1 = F1(ρ, ϕ).data
        ψ2 = F2(ρ, ϕ).data
        1 - real(fidelity(ψ1, ψ2))
    end
end

function H_odf(ρ, ϕ, t, zernike_coefficients_even, zernike_coefficients_odd, U, ψ, μ, ω)
    eventuples, oddtuples = unpack_zernike(zernike_coefficients_even, zernike_coefficients_odd, ρ, ω, t)
    eventerm = oddterm = 0
    if length(eventuples) > 0
        eventerm = mapreduce(x->x[2]*cos(x[1]*ϕ + x[3]), +, eventuples)
    end
    if length(oddtuples) > 0
        oddterm = mapreduce(x->x[2]*sin(x[1]*ϕ + x[3]), +, oddtuples)
    end
    U * cos(-μ*t + ψ + eventerm + oddterm) 
end

function H_odf_sum(ρ, ϕ, t, zernike_coefficients_even, zernike_coefficients_odd, U, ψ, μs, ω)
    eventuples, oddtuples = unpack_zernike(zernike_coefficients_even, zernike_coefficients_odd, ρ, ω, t)
    eventerm = oddterm = 0
    if length(eventuples) > 0
        eventerm = mapreduce(x->x[2]*cos(x[1]*ϕ + x[3]), +, eventuples)
    end
    if length(oddtuples) > 0
        oddterm = mapreduce(x->x[2]*sin(x[1]*ϕ + x[3]), +, oddtuples)
    end
    sum([U * cos(-μ*t + ψ + eventerm + oddterm) for μ in μs]) 
end

function least_squares_solution(mat, y)
    inv(transpose(mat) * mat)*transpose(mat) * y
end

toarray(s::Union{Set, Array}) = [toarray.(s)...]
toarray(v::Number) = v

toarray (generic function with 2 methods)

In [2]:
function annulus(a, b)
    function rtn(ρ, ϕ)
        if a < ρ < b
            1
        else
            0
        end
    end
end

annulus (generic function with 1 method)

In [3]:
function sequential_exact_evolution_evaluator_factory(ψ0, T, zernikeeven, zernikeodd, U, θ, ω, b)
    """Apply all the zernike coefficients given, in order, for time T each."""
    orders = range(0, max(length(zernikeeven), length(zernikeodd)) - 1, step=1)
    function evaluator(ρ, ϕ)
        ψ = ψ0
        # Note that if I redefine orders here, it messes everything up. I don't understand julia scope.
        H(t, _) = H_odf_sum(ρ, ϕ, t, zernikeeven, zernikeodd, U, θ, [order * ω for order in orders], ω)*sigmaz(b), [], []
        _, ψ = @skiptimechecks timeevolution.master_dynamic(T, ψ, H)
        ψ = last(ψ)
    end
end

function annulus_spin_profile(ρ, ϕ, b, r1, r2)
    ψ0 = 1/sqrt(2) * (spindown(b) + spinup(b))
    H(t, _) = annulus(r1, r2)(ρ, ϕ) * sigmaz(b), [], []
    evolution_time = π/2
    step_size = evolution_time/1
    T = [0.0:step_size:evolution_time;];
    _, ψ = timeevolution.master_dynamic(T, ψ0, H)
    last(ψ)
end

annulus_spin_profile (generic function with 1 method)

In [ ]:
targets = []
ys = []
for max_order in [20, 30]
    print("Doing $max_order")
    discretization = 50 # We could probably afford to make this smaller
    zernike_samples = []
    ordering = []
    for n in range(0, max_order, step=1)
        for m in range(0, n, step=1)
            if (n-m)%2 == 0
                Zeven(ρ, ϕ) = R(n, m, ρ) * cos(m * ϕ)
                push!(zernike_samples, sample_across_unit_disk(Zeven, discretization)[3][:])
                push!(ordering, (n, m))
                if m !=0
                    Zodd(ρ, ϕ) = R(n, m, ρ) * sin(m * ϕ)
                    push!(zernike_samples, sample_across_unit_disk(Zodd, discretization)[3][:])
                    push!(ordering, (n, -m))
                end
            end
        end
    end
    Z = reduce(hcat, zernike_samples);
    r1 = .45
    r2 = .65
    xx, yy, zz = sample_across_unit_disk(annulus(r1, r2), discretization)
    G = zz[:];
    coefficients = least_squares_solution(Z, G)
    Γ = 1/62
    ω = 2*π*180E3
    θ = -π/2;
    scaled_coefficients = .5 * coefficients
    reconstruction = zeros(discretization*discretization)
    for (i, zernike_sample) in enumerate(zernike_samples)
        reconstruction += scaled_coefficients[i] * zernike_sample
    end
    lookup = Dict(zip(ordering, scaled_coefficients));
    zernike_even_coefficients = []
    zernike_odd_coefficients = []
    curr_even = []
    curr_odd = []
    for n in range(0, max_order, step=1)
        for m in range(0, n, step=1)
            if (n-m)%2 != 0
                push!(curr_even, 0)
                push!(curr_odd, 0)
            else
                if m == 0
                    push!(curr_even, lookup[(n, m)]/2)
                    push!(curr_odd, lookup[(n, -m)]/2)
                else
                    push!(curr_even, lookup[(n, m)])
                    push!(curr_odd, lookup[(n, -m)])
                end
            end
        end
        push!(zernike_even_coefficients, curr_even)
        push!(zernike_odd_coefficients, curr_odd)
        curr_even = []
        curr_odd = []
    end
    b = SpinBasis(1//2)
    ψ0 = 1/sqrt(2) * (spindown(b) + spinup(b))
    evolution_time =  50E-6
    U = π/(evolution_time) * 2
    step_size = evolution_time
    T = [0.0:step_size:evolution_time;];
    sequential_exact_evolution = sequential_exact_evolution_evaluator_factory(ψ0, T, zernike_even_coefficients, zernike_odd_coefficients, U, θ, ω, b)
    max_ρ = 75
    min_ρ = .25
    len = 20
    ρs = range(min_ρ, max_ρ, step=max_ρ/len)
    angle = 0
    target2 = [Real.(expect(sigmax(b), annulus_spin_profile(ρ, angle, b, r1, r2))) for ρ in ρs]
    y2s = [Real.(expect(sigmax(b), sequential_exact_evolution(ρ, angle))) for ρ in ρs];
    push!(targets, target2)
    push!(ys, y2s)
end

Doing 20

┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEqBase/gLFRA/src/integrator_interface.jl:329
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEqBase/gLFRA/src/integrator_interface.jl:329
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEqBase/gLFRA/src/integrator_interface.jl:329
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEqBase/gLFRA/src/integrator_interface.jl:329
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEqBase/gLFRA/src/integrator_interface.jl:329
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEqBase/gLFRA/src/integrator_interface.jl:329
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ DiffEqBase /home/ampolloreno/.julia/packages/DiffEq

In [ ]:
using Plots
max_ρ = 1
min_ρ = .0
len = 20
ρs = range(min_ρ, max_ρ, step=max_ρ/len)
plot(ρs, ys, label=["Gaussian Hamiltonian" "Zernike Reconstruction (Exact Evolution)"], legend=:bottomright, ylabel="<σx>", xlabel="x", title="Comparing Evolution Under a Gaussian Hamiltonian\n to Evolution Under a Zernike-decomposed Hamiltonian")